In [1]:
import numpy as np
from collections import defaultdict

## 读取数据文件，生成 无向图

In [3]:
data = []
with open('Data.txt' ) as fr: 
    for i, line in enumerate(fr.readlines()):
        if i > 2:
            items = []
            lineArr = line.strip().split(' ')
            for item in lineArr:
                try:
                    items.append(int(item))
                except:
                    pass
            data.append(items)
            
Arr = np.array(data)
graph = dict()
for A in list(set(Arr[:,0])):
    mask = Arr[:,0 ] == A
    graph[A] = list(Arr[mask][:,1])

## 找出无向图中 所有节点的连通分量

In [5]:
def iter_dfs(graph ,start):
    S,Q=set(),[]
    Q.append(start)
    while Q:
        u = Q.pop()
        if u in S:
            continue
        S.add(u)
        Q.extend(graph[u])
        yield u

## cc 存放所有节点的连通分量
cc = dict()
for start in graph.keys():
    L = list(iter_dfs(graph, start))
    cc[start] = L

## 以63节点为sample，输出其连通分量
list(iter_dfs(graph, 63))

[63, 64, 99, 98, 66]

## 找出无向图中所有环（cycle）
  * [GitHub参考地址](https://github.com/qpwo/python-simple-cycles)：https://github.com/qpwo/python-simple-cycles

In [14]:
# Luke Miles, September 2017
# A modification of networkx's implementation of Johnson's cycle finding algorithm
# Original implementation: https://gist.github.com/qpwo/44b48595c2946bb8f823e2d72f687cd8
# Original paper: Donald B Johnson. "Finding all the elementary circuits of a directed graph." SIAM Journal on Computing. 1975.

from collections import defaultdict

def simple_cycles(G):
    # Yield every elementary cycle in python graph G exactly once
    # Expects a dictionary mapping from vertices to iterables of vertices
    def _unblock(thisnode, blocked, B):
        stack = set([thisnode])
        while stack:
            node = stack.pop()
            if node in blocked:
                blocked.remove(node)
                stack.update(B[node])
                B[node].clear()
    G = {v: set(nbrs) for (v,nbrs) in G.items()} # make a copy of the graph
    sccs = strongly_connected_components(G)
    while sccs:
        scc = sccs.pop()
        startnode = scc.pop()
        path=[startnode]
        blocked = set()
        closed = set()
        blocked.add(startnode)
        B = defaultdict(set)
        stack = [ (startnode,list(G[startnode])) ]
        while stack:
            thisnode, nbrs = stack[-1]
            if nbrs:
                nextnode = nbrs.pop()
                if nextnode == startnode:
                    yield path[:]
                    closed.update(path)
                elif nextnode not in blocked:
                    path.append(nextnode)
                    stack.append( (nextnode,list(G[nextnode])) )
                    closed.discard(nextnode)
                    blocked.add(nextnode)
                    continue
            if not nbrs:
                if thisnode in closed:
                    _unblock(thisnode,blocked,B)
                else:
                    for nbr in G[thisnode]:
                        if thisnode not in B[nbr]:
                            B[nbr].add(thisnode)
                stack.pop()
                path.pop()
        remove_node(G, startnode)
        H = subgraph(G, set(scc))
        sccs.extend(strongly_connected_components(H))

def strongly_connected_components(graph):
    # Tarjan's algorithm for finding SCC's
    # Robert Tarjan. "Depth-first search and linear graph algorithms." SIAM journal on computing. 1972.
    # Code by Dries Verdegem, November 2012
    # Downloaded from http://www.logarithmic.net/pfh/blog/01208083168

    index_counter = [0]
    stack = []
    lowlink = {}
    index = {}
    result = []
    
    def _strong_connect(node):
        index[node] = index_counter[0]
        lowlink[node] = index_counter[0]
        index_counter[0] += 1
        stack.append(node)
    
        successors = graph[node]
        for successor in successors:
            if successor not in index:
                _strong_connect(successor)
                lowlink[node] = min(lowlink[node],lowlink[successor])
            elif successor in stack:
                lowlink[node] = min(lowlink[node],index[successor])

        if lowlink[node] == index[node]:
            connected_component = []

            while True:
                successor = stack.pop()
                connected_component.append(successor)
                if successor == node: break
            result.append(connected_component[:])
    
    for node in graph:
        if node not in index:
            _strong_connect(node)
    
    return result

def remove_node(G, target):
    # Completely remove a node from the graph
    # Expects values of G to be sets
    del G[target]
    for nbrs in G.values():
        nbrs.discard(target)

def subgraph(G, vertices):
    # Get the subgraph of G induced by set vertices
    # Expects values of G to be sets
    return {v: G[v] & vertices for v in vertices}

##example:
#graph = {0: [7, 3, 5], 1: [2], 2: [7, 1], 3: [0, 5], 4: [6, 8], 5: [0, 3, 7], 6: [4, 8], 7: [0, 2, 5, 8], 8: [4, 6, 7]}
#print(tuple(simple_cycles(graph)))

cycles = list(simple_cycles(graph))  
cycles[0]  ## 第一个环


[81, 83, 70, 95, 49, 54, 56, 61, 31, 53, 88, 24, 42, 50]